[Back](https://keqideng.github.io/data_analysis_portfolio_project/)
# Chinese Eateries Analysis in U.S.
Date: Sept 23, 2021
Prepared by ***Keqi Deng***

>This dataset is a subset of Yelp’s businesses, reviews, and user data. It was originally put together for the Yelp Dataset Challenge which is a chance for students to conduct research or analysis on Yelp’s data and share their discoveries. In the dataset you’ll find information about businesses across 11 metropolitan areas in four countries.

In [89]:
import gc # garbage collector
import numpy as np # linear algebra
from collections import Counter # for counting commong words
import requests
# request data from yelp
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # visualization
plt.style.use('fivethirtyeight') # use ggplot ploting style
import seaborn as sns # visualization
from wordcloud import WordCloud, STOPWORDS # this module is for making wordcloud in python
import re # regular expression
import string # for finding punctuation in text
import nltk # preprocessing text
from textblob import TextBlob
# import ploty for visualization
import plotly
import plotly.offline as py # make offline
py.init_notebook_mode(connected=True)
import plotly.tools as tls
import plotly.graph_objs as go
from plotly.graph_objs import *
import plotly.tools as tls
import plotly.figure_factory as fig_fact
# remove warnings
import warnings
warnings.filterwarnings('ignore')
# this will allow ploting inside the notebook
%matplotlib inline

## Data Source
The data used in this analysis is provided by [Yelp Opendata](https://www.yelp.com/dataset), last update on Jan 21, 2021.

In [90]:
#Business Search Yelp 'https://api.yelp.com/v3/businesses/search'
client_id = 'By4LztDH6ULQMoOWW73Uag'
api_key = 'hC90EyLZwRlSqQGxIzbg_t8CAfz0aFyIovi270VIjCqEGHBx8xXyPGW4HUQoIh2ELaArE7Lw3zIM9aLkJQkuREJs5S4G1HtBx43eFCpvnP8IebL-uZk2CBKfFqxQYXYx'
endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s'%api_key}

#Define the parameters
params = {'term':'chinese,restaurants',
          'limit':50,
          'radius':10000,
          'location':'NYC'}

#Make a request to the yelp api
response = requests.get(url=endpoint, params=params, headers=headers)

In [91]:
#Convert json string to dictionary
business_data = response.json()
df = pd.DataFrame(business_data['businesses'])
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,0CjK3esfpFcxIopebzjFxA,joes-shanghai-new-york-2,Joe's Shanghai,https://s3-media4.fl.yelpcdn.com/bphoto/KRvabB...,False,https://www.yelp.com/biz/joes-shanghai-new-yor...,6433,"[{'alias': 'shanghainese', 'title': 'Shanghain...",4.0,"{'latitude': 40.7156608, 'longitude': -73.9967...","[pickup, delivery]",$$,"{'address1': '46 Bowery St', 'address2': '', '...",+12122338888,(212) 233-8888,1153.948239
1,wfhMIiMh2A2psCGONOVSlg,han-dynasty-new-york,Han Dynasty,https://s3-media3.fl.yelpcdn.com/bphoto/G5jm9G...,False,https://www.yelp.com/biz/han-dynasty-new-york?...,1273,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...",4.0,"{'latitude': 40.732072, 'longitude': -73.988074}","[pickup, delivery]",$$,"{'address1': '90 3rd Ave', 'address2': '', 'ad...",+12123908685,(212) 390-8685,3006.833805
2,0dDCDQz7DrKkSvu9h2hcQw,chow-house-new-york,Chow House,https://s3-media3.fl.yelpcdn.com/bphoto/kzeCls...,False,https://www.yelp.com/biz/chow-house-new-york?a...,455,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...",4.5,"{'latitude': 40.72915, 'longitude': -74.00072}","[delivery, pickup, restaurant_reservation]",$$,"{'address1': '181 Bleecker St', 'address2': No...",+12128371021,(212) 837-1021,2692.440180
3,lynQoI3w_pzYfHGeuUU-Qg,shu-jiao-fu-zhou-cuisine-restaurant-new-york,Shu Jiao Fu Zhou Cuisine Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/EV10p_...,False,https://www.yelp.com/biz/shu-jiao-fu-zhou-cuis...,910,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.5,"{'latitude': 40.7174109, 'longitude': -73.9920...","[delivery, pickup]",$,"{'address1': '295 Grand St', 'address2': None,...",+12126252532,(212) 625-2532,1343.485119
4,xq0cX_DgxiJMXwhmEl9kUA,café-china-new-york-2,Café China,https://s3-media1.fl.yelpcdn.com/bphoto/UhHqxE...,False,https://www.yelp.com/biz/caf%C3%A9-china-new-y...,1693,"[{'alias': 'szechuan', 'title': 'Szechuan'}]",4.0,"{'latitude': 40.75157, 'longitude': -73.98584}","[pickup, delivery]",$$,"{'address1': '59 West 37th St', 'address2': No...",+12122132810,(212) 213-2810,5186.281754


In [92]:
coordinates = df.coordinates
lat = [i['latitude'] for i in coordinates]
lon = [i['longitude'] for i in coordinates]
df['latitude'] = lat
df['longitude'] = lon
df.drop('coordinates',axis=1,inplace=True)

In [93]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,0CjK3esfpFcxIopebzjFxA,joes-shanghai-new-york-2,Joe's Shanghai,https://s3-media4.fl.yelpcdn.com/bphoto/KRvabB...,False,https://www.yelp.com/biz/joes-shanghai-new-yor...,6433,"[{'alias': 'shanghainese', 'title': 'Shanghain...",4.0,"[pickup, delivery]",$$,"{'address1': '46 Bowery St', 'address2': '', '...",+12122338888,(212) 233-8888,1153.948239,40.715661,-73.996701
1,wfhMIiMh2A2psCGONOVSlg,han-dynasty-new-york,Han Dynasty,https://s3-media3.fl.yelpcdn.com/bphoto/G5jm9G...,False,https://www.yelp.com/biz/han-dynasty-new-york?...,1273,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...",4.0,"[pickup, delivery]",$$,"{'address1': '90 3rd Ave', 'address2': '', 'ad...",+12123908685,(212) 390-8685,3006.833805,40.732072,-73.988074
2,0dDCDQz7DrKkSvu9h2hcQw,chow-house-new-york,Chow House,https://s3-media3.fl.yelpcdn.com/bphoto/kzeCls...,False,https://www.yelp.com/biz/chow-house-new-york?a...,455,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...",4.5,"[delivery, pickup, restaurant_reservation]",$$,"{'address1': '181 Bleecker St', 'address2': No...",+12128371021,(212) 837-1021,2692.440180,40.729150,-74.000720
3,lynQoI3w_pzYfHGeuUU-Qg,shu-jiao-fu-zhou-cuisine-restaurant-new-york,Shu Jiao Fu Zhou Cuisine Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/EV10p_...,False,https://www.yelp.com/biz/shu-jiao-fu-zhou-cuis...,910,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.5,"[delivery, pickup]",$,"{'address1': '295 Grand St', 'address2': None,...",+12126252532,(212) 625-2532,1343.485119,40.717411,-73.992088
4,xq0cX_DgxiJMXwhmEl9kUA,café-china-new-york-2,Café China,https://s3-media1.fl.yelpcdn.com/bphoto/UhHqxE...,False,https://www.yelp.com/biz/caf%C3%A9-china-new-y...,1693,"[{'alias': 'szechuan', 'title': 'Szechuan'}]",4.0,"[pickup, delivery]",$$,"{'address1': '59 West 37th St', 'address2': No...",+12122132810,(212) 213-2810,5186.281754,40.751570,-73.985840


In [94]:
yelp_df = df
yelp_df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,0CjK3esfpFcxIopebzjFxA,joes-shanghai-new-york-2,Joe's Shanghai,https://s3-media4.fl.yelpcdn.com/bphoto/KRvabB...,False,https://www.yelp.com/biz/joes-shanghai-new-yor...,6433,"[{'alias': 'shanghainese', 'title': 'Shanghain...",4.0,"[pickup, delivery]",$$,"{'address1': '46 Bowery St', 'address2': '', '...",+12122338888,(212) 233-8888,1153.948239,40.715661,-73.996701
1,wfhMIiMh2A2psCGONOVSlg,han-dynasty-new-york,Han Dynasty,https://s3-media3.fl.yelpcdn.com/bphoto/G5jm9G...,False,https://www.yelp.com/biz/han-dynasty-new-york?...,1273,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...",4.0,"[pickup, delivery]",$$,"{'address1': '90 3rd Ave', 'address2': '', 'ad...",+12123908685,(212) 390-8685,3006.833805,40.732072,-73.988074


In [95]:
# Dataset information
yelp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             50 non-null     object 
 1   alias          50 non-null     object 
 2   name           50 non-null     object 
 3   image_url      50 non-null     object 
 4   is_closed      50 non-null     bool   
 5   url            50 non-null     object 
 6   review_count   50 non-null     int64  
 7   categories     50 non-null     object 
 8   rating         50 non-null     float64
 9   transactions   50 non-null     object 
 10  price          44 non-null     object 
 11  location       50 non-null     object 
 12  phone          50 non-null     object 
 13  display_phone  50 non-null     object 
 14  distance       50 non-null     float64
 15  latitude       50 non-null     float64
 16  longitude      50 non-null     float64
dtypes: bool(1), float64(4), int64(1), object(11)
memory usag

In [96]:
# Dataset length:
len(yelp_df)

50

## Dataset Modification
> Modify the Dataset to Find Restaurants Labeled *Chinese*, then plot the location of Chinese restaurants on the map.

In [97]:
lat_mean = yelp_df.latitude.mean()
lon_mean = yelp_df.longitude.mean()

In [98]:
# import plotly for plotting interactive map
hov_label = yelp_df['name']

# use mapbox token
tkn = 'pk.eyJ1IjoicGF0cmlja2RkZCIsImEiOiJja3R5dGluOWEzNzE3MzFvMzR0MjRlZWVtIn0.ezHYtubmTTHs1z2n11c7yQ'

data = Data([Scattermapbox(lat=yelp_df.latitude.tolist(),
                           lon=yelp_df.longitude.tolist(),
                                       mode='markers',
                                       text=hov_label)])
layout = Layout(title='Chinese Restaurants on Yelp',
                autosize=True,
                hovermode='closest',
                mapbox=dict(accesstoken=tkn,
                            bearing=0,
                            center=dict(lat=lat_mean, lon=lon_mean),
                            style='light',
                            pitch=0,
                            zoom=10))

fig=dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename='mapbox')